# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [228]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact

In [229]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAP_BOX_API")
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [230]:
# Import the CSVs to Pandas DataFrames
file_path = Path("C:/Repositories/Homework5_PyViz/homework-pyviz/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("C:/Repositories/Homework5_PyViz/homework-pyviz/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [231]:
# Define Panel Visualization Functions

def housing_units_per_year():
    
    grouped_df = sfo_data.groupby(level=0).mean()
    grouped_df.drop(columns=['sale_price_sqr_foot', 'gross_rent'], inplace=True)
    
    plot_1 = grouped_df.hvplot.bar(figsize=(10, 5), ylim=(370000, 390000), title="Housing Units Per Year", yformatter='%.0f')
    
    return pn.panel(plot_1)

def average_gross_rent():
    
    units_by_year = sfo_data.groupby(level=0).count()
    units_by_year.drop(columns=['neighborhood', 'sale_price_sqr_foot'], inplace=True)
    grossrent_by_year = sfo_data.groupby(level=0).sum()
    units_by_year['gross_rent'] = grossrent_by_year['gross_rent']
    units_by_year['average_rent'] = units_by_year['gross_rent'] / units_by_year['housing_units'].copy()
    
    average_rent_by_year = pd.DataFrame()
    average_rent_by_year.reindex_like(units_by_year)
    average_rent_by_year['average_rent'] = units_by_year['average_rent']
    
    plot_2 = average_rent_by_year.hvplot.line(figsize=(10, 5), title="Average Rent Per Year")
    
    return pn.panel(plot_2)

def average_sales_price():
    
    units_by_year = sfo_data.groupby(level=0).count()
    units_by_year_2 = sfo_data.groupby(level=0).count()
    units_by_year_2.drop(columns=['neighborhood', 'gross_rent'], inplace=True)
    sales_price = sfo_data.groupby(level=0).sum()
    units_by_year_2['sale_price_sqr_foot'] = sales_price['sale_price_sqr_foot']
    units_by_year_2['average_sale_price_sqr_ft'] = units_by_year_2['sale_price_sqr_foot'] / units_by_year_2['housing_units'].copy()
    
    average_sale_price_per_year = pd.DataFrame()
    average_sale_price_per_year.reindex_like(units_by_year)
    average_sale_price_per_year['average_sale_price_sqr_ft'] = units_by_year_2['average_sale_price_sqr_ft']
    
    plot_3 = average_sale_price_per_year.hvplot.line(figsize=(10, 5), title="Average Sale Price Per Square Foot (Annual)")
   
    return pn.panel(plot_3)

def average_price_by_neighborhood():
    
    grouped_multiple = sfo_data.groupby(['year', 'neighborhood']).mean()
    grouped_multiple.drop(columns=['housing_units', 'gross_rent'], inplace=True)
    
    plot_4 = grouped_multiple.hvplot.line(x='year', y='sale_price_sqr_foot', groupby='neighborhood')
    
    return pn.panel(plot_4)

### ^ having difficulty with the dynamic map

def top_most_expensive_neighborhoods():
    
    most_expensive = sfo_data.groupby(['neighborhood']).mean()
    most_expensive.drop(columns=['housing_units', 'gross_rent'], inplace=True)
    most_expensive.sort_values(by=['sale_price_sqr_foot'], ascending=False, inplace=True)
    most_expensive_plot = most_expensive.head(10)

    plot_5 = most_expensive_plot.hvplot.bar(figsize=(10, 5), title="10 Most Expensive Neighborhoods", rot=45)

    return pn.panel(plot_5)

def parallel_coordinates():
    
    parallel_coordinates_df = sfo_data.groupby(['neighborhood']).mean()
    pcdf2 = pd.DataFrame(columns=['sale_price_sqr_foot', 'housing_units', 'gross_rent'], index=None)
    pcdf2['sale_price_sqr_foot'] = parallel_coordinates_df['sale_price_sqr_foot']
    pcdf2['housing_units'] = parallel_coordinates_df['housing_units']
    pcdf2['gross_rent'] = parallel_coordinates_df['gross_rent']
    pcdf2.sort_values(by=['sale_price_sqr_foot'], ascending=False, inplace=True)
    pcdf2_plot = pcdf2.head(10)
    pcdf2_plot.reset_index(drop=True, inplace=True)
    
    plot_6 = px.parallel_coordinates(pcdf2_plot, color='sale_price_sqr_foot')
    
    return pn.panel(plot_6)

def parallel_categories():
    
    parallel_categories_df = sfo_data.groupby(['neighborhood']).mean()
    parallel_categories_df.sort_values(by=['sale_price_sqr_foot'], ascending=False, inplace=True)
    parallel_categories_df.reset_index(inplace=True)
    pcdf3_plot = parallel_categories_df.head(10)
    
    plot_7 = px.parallel_categories(pcdf3_plot, color='sale_price_sqr_foot',)
    
    return pn.panel(plot_7)

def neighborhood_map():
    
    file_path = Path("C:/Repositories/Homework5_PyViz/homework-pyviz/neighborhoods_coordinates.csv")
    df_neighborhood_locations = pd.read_csv(file_path)
    df_neighborhood_locations.columns = ['neighborhood', 'Lat', 'Lon']
    neighborhood_means = sfo_data.groupby(['neighborhood']).mean()
    neighborhood_means.reset_index(inplace=True)
    map_concat_df = pd.concat([df_neighborhood_locations, neighborhood_means], axis="columns", join="inner")
    map_concat_df.columns = ['neighborhood', 'Latitude', 'Longitude', 'drop_1', 'Sales Price Sqr Foot', 'drop_2', 'Gross Rent'] 
    map_concat_df.drop(columns=['drop_1', 'drop_2'], inplace=True)
    
    plot_8 = px.scatter_mapbox(map_concat_df, lat="Latitude", lon="Longitude", color="Gross Rent")
    
    return pn.panel(plot_8)


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [232]:
row_1 = pn.Row(housing_units_per_year())
row_2 = pn.Row(average_gross_rent(), average_sales_price())
row_3 = pn.Row(average_price_by_neighborhood(), top_most_expensive_neighborhoods())
row_4 = pn.Row(parallel_coordinates(), parallel_categories())
row_5 = pn.Row(neighborhood_map())

column_1 = pn.Column('Housing Units Per Year', row_1)
column_2 = pn.Column('Average Gross Rent & Sale Price', row_2)
column_3 = pn.Column('Neighborhood Data: Average Price & Most Expensive', row_3)
column_4 = pn.Column('Unique Data Graphs', row_4)
column_5 = pn.Column('Geographic Map', row_5)

panel = pn.Tabs(
    ('Basic Data', column_1), 
    ('Average Data', column_2), 
    ('Neighborhood Data', column_3), 
    ('Unique Graphs', column_4), 
    ('Geographic Map', column_5)
)

## Serve the Panel Dashboard

In [233]:
panel.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(Bars)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(Curve)
            [1] HoloViews(Curve)
    [2] Column
        [0] Markdown(str)
        [1] Row
            [0] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
            [1] HoloViews(Bars)
    [3] Column
        [0] Markdown(str)
        [1] Row
            [0] Plotly(Figure)
            [1] Plotly(Figure)
    [4] Column
        [0] Markdown(str)
        [1] Row
            [0] Plotly(Figure)

In [ ]:
# done